In [1]:
%run to_import.py
%matplotlib inline

In [2]:
segm_set=range(1,13)
data_folder='../../Data_Raw'
data_small='../../Data_Small_Tables'
replicas_all=range(13)
for segment in segm_set:
    replicas_all[segment]=pd.read_csv(data_folder+'/S'+str(segment)+'_merged_filtered_sum_10_size_4_with_fr.csv',sep='\t',index_col=0)

In [4]:
#download k and s12 parameters
table_s12_k=pd.read_csv(data_small+'/table_s12_k_with_nons.csv',sep='\t',index_col=0)

## group by aa

In [5]:
#merge
aa_all=range(13)
for s in segm_set:
    replica=replicas_all[s]
    by_aa =replica[['aa_seq','t0_fr','t1_fr','t2_fr','t0_fr_var','t1_fr_var','t2_fr_var']].\
    groupby('aa_seq',as_index=False)
    aa_all[s]=by_aa.sum()
    aa_all[s]['size']=by_aa.size().values
    print s,len(replica),len(aa_all[s])

1 515086 126100
2 419808 123388
3 685432 95148
4 468060 69253
5 718331 100545
6 257527 84052
7 177617 26377
8 198701 73844
9 166104 88766
10 275314 80780
11 177863 37579
12 277175 63889


## fit all data

In [7]:
#calsualte fit for each point 
for s in segm_set:
    print 'S'+str(s)
    data=aa_all[s][['t0_fr','t1_fr','t2_fr']].values.copy()
    data_sigma=np.sqrt(aa_all[s][['t0_fr_var','t1_fr_var','t2_fr_var']].values.copy())
    
    s12=table_s12_k.loc['S'+str(s),'s12']
    k=table_s12_k.loc['S'+str(s),'k']
    
    y0,si,y0_std,s_std,residuals=func_fit_single_exponents(data,data_sigma,s12,k)
    aa_all[s]['y0']=y0
    aa_all[s]['s']=si
    aa_all[s]['y0_std']=y0_std
    aa_all[s]['s_std']=s_std
    aa_all[s]['residuals']=residuals
    aa_all[s].to_csv(data_folder+'/S'+str(s)+'_sum_10_size_4_with_nons_aa_avg'+'.csv',sep='\t')

S1
S2
S3
S4
S5
S6
S7
S8
S9
S10
S11
S12


## Boundary

In [9]:
readsum=pd.read_csv(data_small+'/readsum_all.csv', sep='\t',index_col=0)

There are some genotypes with t1==0 and t0,t2!=0. For all segments there amount is neglectible ~ 0-100, only for S9 there are ~4000 of them. s_std>0.5 or nan (because fit failed) for each of these examples.
I can't find any better solution than just filter all them out.

In [10]:
#delete all genotypes with 001,010,101
for s in segm_set:
    index001=aa_all[s][(aa_all[s]['t0_fr']==0)&(aa_all[s]['t1_fr']==0)&(aa_all[s]['t2_fr']!=0)].index
    aa_all[s].drop(aa_all[s].index[index001],inplace=True)
    aa_all[s].reset_index(drop=True,inplace=True)
    index010=aa_all[s][(aa_all[s]['t0_fr']==0)&(aa_all[s]['t1_fr']!=0)&(aa_all[s]['t2_fr']==0)].index
    aa_all[s].drop(aa_all[s].index[index010],inplace=True)
    aa_all[s].reset_index(drop=True,inplace=True)
    index101=aa_all[s][(aa_all[s]['t0_fr']!=0)&(aa_all[s]['t1_fr']==0)&(aa_all[s]['t2_fr']!=0)].index
    aa_all[s].drop(aa_all[s].index[index101],inplace=True)
    aa_all[s].reset_index(drop=True,inplace=True)
    print 'S'+str(s),(len(index001)),(len(index010)),(len(index101))

S1 0 0 13
S2 3 2 156
S3 4 7 31
S4 0 0 8
S5 1 0 23
S6 0 0 14
S7 0 0 0
S8 1 1 36
S9 213 36 3734
S10 40 45 260
S11 14 25 118
S12 4 2 46


In [11]:
#for 100 boundary=1, the rest are 011, 110, 111 for them boundary=0 and we just use fit
for s in segm_set:
    aa_all[s]['boundary']=0
    index100=aa_all[s][(aa_all[s]['t0_fr']!=0)&(aa_all[s]['t1_fr']==0)&(aa_all[s]['t2_fr']==0)].index
    aa_all[s].loc[index100,'boundary']=1

In [12]:
# for 001 (boundary==1) calculate max fitness
for s in segm_set:
    readsum_max_t1=max([readsum.loc['t1_tech1_bio1','S'+str(s)],readsum.loc['t1_tech1_bio2','S'+str(s)],\
                        readsum.loc['t1_tech2_bio1','S'+str(s)],readsum.loc['t1_tech2_bio2','S'+str(s)]])
    aa_all[s]['s']=\
    np.where(aa_all[s]['boundary']==0,aa_all[s]['s'],np.log((10.**6/readsum_max_t1)/aa_all[s]['t0_fr']))

In [13]:
for s in segm_set:
    print s, len(aa_all[s][aa_all[s]['boundary']==1])

1 95
2 2236
3 106
4 10
5 133
6 672
7 11
8 859
9 11011
10 3540
11 7215
12 562


In [ ]:
#filter that s_std<0.5
#for s in segm_set:
#    aa_all[s]=aa_all[s][(aa_all[s]['boundary']==1)|(aa_all[s]['s_std']<0.5)]

In [15]:
#save
for s in segm_set:
    aa_all[s].to_csv(data_folder+'/S'+str(s)+'_sum_10_size_4_with_nons_aa_avg'+'.csv',sep='\t')